In [1]:
# Import dependencies
import pandas as pd
from datetime import datetime, timedelta
import re
import os
import glob
from functools import reduce

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Concatenating All DataFrames

In [2]:
raceNames = [
    'Bahrain',
    'Saudi Arabia',
    'Australia',
    'Emilia Romagna',
    'Miami',
    'Spain',
    'Monaco',
    'Azerbaijan',
    'Canada',
    'Great Britain',
    'Austria',
    'France',
    'Hungary'
]

In [3]:
master_df = pd.DataFrame()
race_df_list = []

In [4]:
# use glob to get all the csv files 
# in the folder
csv_files = glob.glob(os.path.join('./data', "*.csv"))
csv_files.sort()
csv_files

['./data/00_0_df.csv',
 './data/00_1_df.csv',
 './data/00_2_df.csv',
 './data/00_3_df.csv',
 './data/00_4_df.csv',
 './data/00_5_df.csv',
 './data/00_6_df.csv',
 './data/00_7_df.csv',
 './data/01_0_df.csv',
 './data/01_1_df.csv',
 './data/01_2_df.csv',
 './data/01_3_df.csv',
 './data/01_4_df.csv',
 './data/01_5_df.csv',
 './data/01_6_df.csv',
 './data/01_7_df.csv',
 './data/02_0_df.csv',
 './data/02_1_df.csv',
 './data/02_2_df.csv',
 './data/02_3_df.csv',
 './data/02_4_df.csv',
 './data/02_5_df.csv',
 './data/02_6_df.csv',
 './data/02_7_df.csv',
 './data/03_0_df.csv',
 './data/03_1_df.csv',
 './data/03_2_df.csv',
 './data/03_3_df.csv',
 './data/03_4_df.csv',
 './data/03_5_df.csv',
 './data/03_6_df.csv',
 './data/03_7_df.csv',
 './data/03_8_df.csv',
 './data/04_0_df.csv',
 './data/04_1_df.csv',
 './data/04_2_df.csv',
 './data/04_3_df.csv',
 './data/04_4_df.csv',
 './data/04_5_df.csv',
 './data/04_6_df.csv',
 './data/04_7_df.csv',
 './data/05_0_df.csv',
 './data/05_1_df.csv',
 './data/05

In [5]:
# loop over the list of csv files, converting >100 tables into individual race dataframes
for count, fileLocation in enumerate(csv_files):
      
    # read the csv file
    df = pd.read_csv(fileLocation)
      
    # print the location and filename
    print('Location:', fileLocation)
    fileName = fileLocation.split("/")[-1]
    raceIndex = fileName[0:2]
    tableIndex = int(fileName[3])
    print('File Name:', fileName)
    print(f'Race Number Index: {raceIndex}')
    print(f'Table Number Index: {tableIndex}')
    
    # Races 3 and 10 have sprints and need to be concatted differently
    if (raceIndex == '03') or (raceIndex == '10'):
        if fileName == f'{raceIndex}_0_df.csv':
            master_df['Final Race Position'] = df['Pos']
            master_df['No'] = df['No']
            master_df['Driver Name'] = df['Driver']
            master_df['Car'] = df['Car']
            master_df['Race Laps'] = df['Laps']
            master_df['Race Time'] = df['Time/Retired']
            master_df['Race Points'] = df['PTS']
            master_df['Race'] = raceNames[int(raceIndex)]
            master_df.set_index('No', inplace=True)
        elif fileName == f'{raceIndex}_1_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Lap', 'Time of day', 'Time', 'Avg Speed']], on='No')
            master_df.rename(columns={"Lap": "Fastest Lap No", "Time": "Fastest Lap Time"}, inplace=True)
        elif fileName == f'{raceIndex}_2_df.csv':
            df.set_index('No', inplace=True)
            df.sort_values(by=['Driver', 'Stops'])
            df.drop_duplicates(subset=['Driver'], keep='last', inplace=True)
            master_df = master_df.join(df[['Stops', 'Total']], on='No')
            master_df.rename(columns={"Total": "Total Pit Time"}, inplace=True)
        elif fileName == f'{raceIndex}_3_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos']], on='No')
            master_df.rename(columns={"Pos": "Starting Grid Pos"}, inplace=True)
        elif fileName == f'{raceIndex}_4_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Laps', 'Time/Retired', 'PTS']], on='No')
            master_df.rename(columns={"Pos": "Final Sprint Postition", "Laps": "Sprint Laps", "Time/Retired": "Sprint Time/Retired", "PTS": "Sprint PTS"}, inplace=True)
        elif fileName == f'{raceIndex}_5_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Time']], on='No')
            master_df.rename(columns={"Pos": "Starting Sprint Postition", "Time": "Sprint Grid Time"}, inplace=True)
        elif fileName == f'{raceIndex}_6_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Time', 'Gap', 'Laps']], on='No')
            master_df.rename(columns={"Pos": "P2 Postition", "Time": "P2 Time", "Gap": "P2 Gap", "Laps": "P2 Laps"}, inplace=True)
        elif fileName == f'{raceIndex}_7_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Q1', 'Q2', 'Q3', 'Laps']], on='No')
            master_df.rename(columns={"Pos": "Qualifying Postition", "Laps": "Qualifying Laps"}, inplace=True)
        elif fileName == f'{raceIndex}_8_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Time', 'Gap', 'Laps']], on='No')
            master_df.rename(columns={"Pos": "P1 Postition", "Time": "P1 Time", "Gap": "P1 Gap", "Laps": "P1 Laps"}, inplace=True)
        else:
            print("SOMETHING WENT WRONG")
    else:
        if fileName == f'{raceIndex}_0_df.csv':
            master_df['Final Race Position'] = df['Pos']
            master_df['No'] = df['No']
            master_df['Driver Name'] = df['Driver']
            master_df['Car'] = df['Car']
            master_df['Race Laps'] = df['Laps']
            master_df['Race Time'] = df['Time/Retired']
            master_df['Race Points'] = df['PTS']
            master_df['Race'] = raceNames[int(raceIndex)]
            master_df.set_index('No', inplace=True)
        elif fileName == f'{raceIndex}_1_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Lap', 'Time of day', 'Time', 'Avg Speed']], on='No')
            master_df.rename(columns={"Lap": "Fastest Lap No", "Time": "Fastest Lap Time"}, inplace=True)
        elif fileName == f'{raceIndex}_2_df.csv':
            df.set_index('No', inplace=True)
            df.sort_values(by=['Driver', 'Stops'])
            df.drop_duplicates(subset=['Driver'], keep='last', inplace=True)
            master_df = master_df.join(df[['Stops', 'Total']], on='No')
            master_df.rename(columns={"Time": "Fastest Lap Time"}, inplace=True)
        elif fileName == f'{raceIndex}_3_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Time']], on='No')
            master_df.rename(columns={"Lap": "Fastest Lap No", "Total": "Total Pit Time", "Pos": "Starting Grid Pos", "Time": "Starting Grid Quali Time"}, inplace=True)
        elif fileName == f'{raceIndex}_4_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Q1', 'Q2', 'Q3', 'Laps']], on='No')
            master_df.rename(columns={"Pos": "Qualifying Postition", "Laps": "Qualifying Laps"}, inplace=True)
        elif fileName == f'{raceIndex}_5_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Time', 'Gap', 'Laps']], on='No')
            master_df.rename(columns={"Pos": "P3 Postition", "Time": "P3 Time", "Gap": "P3 Gap", "Laps": "P3 Laps"}, inplace=True)
        elif fileName == f'{raceIndex}_6_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Time', 'Gap', 'Laps']], on='No')
            master_df.rename(columns={"Pos": "P2 Postition", "Time": "P2 Time", "Gap": "P2 Gap", "Laps": "P2 Laps"}, inplace=True)
        elif fileName == f'{raceIndex}_7_df.csv':
            df.set_index('No', inplace=True)
            master_df = master_df.join(df[['Pos', 'Time', 'Gap', 'Laps']], on='No')
            master_df.rename(columns={"Pos": "P1 Postition", "Time": "P1 Time", "Gap": "P1 Gap", "Laps": "P1 Laps"}, inplace=True)
        else:
            print("SOMETHING WENT WRONG")
            
    # The sprint races have one more raceIndex, so the master_df needs to be reset at 7 and 8    
    if (raceIndex != '03' and raceIndex != '10') and (tableIndex == 7):
        master_df.reset_index(inplace=True)
        race_df_list.append(master_df)
        print(f"{raceNames[int(raceIndex)]}_DF has been added successfully")
        master_df = pd.DataFrame()
    elif (tableIndex == 8):
        master_df.reset_index(inplace=True)
        race_df_list.append(master_df)
        print(f"{raceNames[int(raceIndex)]}_DF has been added successfully")
        master_df = pd.DataFrame()
        

Location: ./data/00_0_df.csv
File Name: 00_0_df.csv
Race Number Index: 00
Table Number Index: 0

TABLE INDEX 0
Location: ./data/00_1_df.csv
File Name: 00_1_df.csv
Race Number Index: 00
Table Number Index: 1

TABLE INDEX 1
Location: ./data/00_2_df.csv
File Name: 00_2_df.csv
Race Number Index: 00
Table Number Index: 2

TABLE INDEX 2
Location: ./data/00_3_df.csv
File Name: 00_3_df.csv
Race Number Index: 00
Table Number Index: 3

TABLE INDEX 3
Location: ./data/00_4_df.csv
File Name: 00_4_df.csv
Race Number Index: 00
Table Number Index: 4

TABLE INDEX 4
Location: ./data/00_5_df.csv
File Name: 00_5_df.csv
Race Number Index: 00
Table Number Index: 5

TABLE INDEX 5
Location: ./data/00_6_df.csv
File Name: 00_6_df.csv
Race Number Index: 00
Table Number Index: 6

TABLE INDEX 6
Location: ./data/00_7_df.csv
File Name: 00_7_df.csv
Race Number Index: 00
Table Number Index: 7

RESET MASTER DF FOR REG RACES
    No Final Race Position             Driver Name  \
0   16                   1   Charles  Lecl

TABLE INDEX 1
Location: ./data/04_2_df.csv
File Name: 04_2_df.csv
Race Number Index: 04
Table Number Index: 2

TABLE INDEX 2
Location: ./data/04_3_df.csv
File Name: 04_3_df.csv
Race Number Index: 04
Table Number Index: 3

TABLE INDEX 3
Location: ./data/04_4_df.csv
File Name: 04_4_df.csv
Race Number Index: 04
Table Number Index: 4

TABLE INDEX 4
Location: ./data/04_5_df.csv
File Name: 04_5_df.csv
Race Number Index: 04
Table Number Index: 5

TABLE INDEX 5
Location: ./data/04_6_df.csv
File Name: 04_6_df.csv
Race Number Index: 04
Table Number Index: 6

TABLE INDEX 6
Location: ./data/04_7_df.csv
File Name: 04_7_df.csv
Race Number Index: 04
Table Number Index: 7

RESET MASTER DF FOR REG RACES
    No Final Race Position             Driver Name  \
0    1                   1    Max  Verstappen  VER   
1   16                   2   Charles  Leclerc  LEC   
2   55                   3      Carlos  Sainz  SAI   
3   11                   4      Sergio  Perez  PER   
4   63                   5    Geor

TABLE INDEX 0
Location: ./data/08_1_df.csv
File Name: 08_1_df.csv
Race Number Index: 08
Table Number Index: 1

TABLE INDEX 1
Location: ./data/08_2_df.csv
File Name: 08_2_df.csv
Race Number Index: 08
Table Number Index: 2

TABLE INDEX 2
Location: ./data/08_3_df.csv
File Name: 08_3_df.csv
Race Number Index: 08
Table Number Index: 3

TABLE INDEX 3
Location: ./data/08_4_df.csv
File Name: 08_4_df.csv
Race Number Index: 08
Table Number Index: 4

TABLE INDEX 4
Location: ./data/08_5_df.csv
File Name: 08_5_df.csv
Race Number Index: 08
Table Number Index: 5

TABLE INDEX 5
Location: ./data/08_6_df.csv
File Name: 08_6_df.csv
Race Number Index: 08
Table Number Index: 6

TABLE INDEX 6
Location: ./data/08_7_df.csv
File Name: 08_7_df.csv
Race Number Index: 08
Table Number Index: 7

RESET MASTER DF FOR REG RACES
    No Final Race Position             Driver Name  \
0    1                   1    Max  Verstappen  VER   
1   55                   2      Carlos  Sainz  SAI   
2   44                   3    L

RESET MASTER DF FOR SPRINT RACES
    No Final Race Position             Driver Name  \
0   16                   1   Charles  Leclerc  LEC   
1    1                   2    Max  Verstappen  VER   
2   44                   3    Lewis  Hamilton  HAM   
3   63                   4    George  Russell  RUS   
4   31                   5      Esteban  Ocon  OCO   
5   47                   6   Mick  Schumacher  MSC   
6    4                   7      Lando  Norris  NOR   
7   20                   8   Kevin  Magnussen  MAG   
8    3                   9  Daniel  Ricciardo  RIC   
9   14                  10   Fernando  Alonso  ALO   
10  77                  11   Valtteri  Bottas  BOT   
11  23                  12   Alexander  Albon  ALB   
12  18                  13      Lance  Stroll  STR   
13  24                  14       Zhou  Guanyu  ZHO   
14  10                  15      Pierre  Gasly  GAS   
15  22                  16      Yuki  Tsunoda  TSU   
16   5                  17  Sebastian  Vettel  VE

In [6]:
# Concat the race DataFrames into one F1 DataFrame
f1_df = pd.concat(race_df_list, axis=0, ignore_index=True)

In [8]:
# Set index to the Race and Driver No
f1_df.set_index(['Race', 'No'], inplace=False)

Final Race Position             Driver Name  \
Race           No                                               
Bahrain        16                   1   Charles  Leclerc  LEC   
               55                   2      Carlos  Sainz  SAI   
               44                   3    Lewis  Hamilton  HAM   
               63                   4    George  Russell  RUS   
               20                   5   Kevin  Magnussen  MAG   
               77                   6   Valtteri  Bottas  BOT   
               31                   7      Esteban  Ocon  OCO   
               22                   8      Yuki  Tsunoda  TSU   
               14                   9   Fernando  Alonso  ALO   
               24                  10       Zhou  Guanyu  ZHO   
               47                  11   Mick  Schumacher  MSC   
               18                  12      Lance  Stroll  STR   
               23                  13   Alexander  Albon  ALB   
               3                   14  Daniel  Ricciardo  RIC   
               4                   15      Lando  Norris  NOR   
               6                   16   Nicholas  Latifi  LAT   
               27                  17   Nico  Hulkenberg  HUL   
               11                  18      Sergio  Perez  PER   
               1                   19    Max  Verstappen  VER   
               10                  NC      Pierre  Gasly  GAS   
Saudi Arabia   1                    1    Max  Verstappen  VER   
               16                   2   Charles  Leclerc  LEC   
               55                   3      Carlos  Sainz  SAI   
               11                   4      Sergio  Perez  PER   
               63                   5    George  Russell  RUS   
               31                   6      Esteban  Ocon  OCO   
               4                    7      Lando  Norris  NOR   
               10                   8      Pierre  Gasly  GAS   
               20                   9   Kevin  Magnussen  MAG   
               44                  10    Lewis  Hamilton  HAM   
               24                  11       Zhou  Guanyu  ZHO   
               27                  12   Nico  Hulkenberg  HUL   
               18                  13      Lance  Stroll  STR   
               23                  14   Alexander  Albon  ALB   
               77                  NC   Valtteri  Bottas  BOT   
               14                  NC   Fernando  Alonso  ALO   
               3                   NC  Daniel  Ricciardo  RIC   
               6                   NC   Nicholas  Latifi  LAT   
               22                  NC      Yuki  Tsunoda  TSU   
               47                  NC   Mick  Schumacher  MSC   
Australia      16                   1   Charles  Leclerc  LEC   
               11                   2      Sergio  Perez  PER   
               63                   3    George  Russell  RUS   
               44                   4    Lewis  Hamilton  HAM   
               4                    5      Lando  Norris  NOR   
               3                    6  Daniel  Ricciardo  RIC   
               31                   7      Esteban  Ocon  OCO   
               77                   8   Valtteri  Bottas  BOT   
               10                   9      Pierre  Gasly  GAS   
               23                  10   Alexander  Albon  ALB   
               24                  11       Zhou  Guanyu  ZHO   
               18                  12      Lance  Stroll  STR   
               47                  13   Mick  Schumacher  MSC   
               20                  14   Kevin  Magnussen  MAG   
               22                  15      Yuki  Tsunoda  TSU   
               6                   16   Nicholas  Latifi  LAT   
               14                  17   Fernando  Alonso  ALO   
               1                   NC    Max  Verstappen  VER   
               5                   NC  Sebastian  Vettel  VET   
               55                  NC      Carlos  Sa

In [10]:
# Save f1_df to csv file
# f1_df.to_csv(f'./data/f1_df.csv', index=False)

# Data Cleaning

In [ ]:
# Begin data cleaning